# Model Loading
Notebook with scripts to download local LLMs for dev testing the Streamlit app. This skips the download process that
would've occurred on the first run-through of the Streamlit app scripts.

In [1]:
import torch
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    # model="GeneZC/MiniChat-2-3B",
    model="vihangd/DopeyTinyLlama-1.1B-v1",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)
llm = HuggingFacePipeline(pipeline=pipe)

/home/alvin/miniconda3/envs/lc-update/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import (
    RunnableParallel,
    RunnablePassthrough,
    RunnableLambda,
)

In [3]:
output_parser = StrOutputParser()
beef_recipe = """Recipe Name: Beef Stroganoff, 
Ingredients: 
1 pound ground beef, 
0.5 cup chopped onion, 
1 tablespoon all-purpose flour, 
0.5 teaspoon salt, 
0.25 teaspoon paprika, 
1 (10.75 ounce) can condensed cream of mushroom soup, 
1 cup sour cream, 
8 ounces egg noodles, 

Cooking Instructions: 
1. In a large skillet over medium heat, sauté beef and onions for 10 minutes, or until meat is browned and onion is tender.
2. Stir in flour, salt, and paprika. Add condensed soup, mix well, and cook, uncovered, for 20 minutes.
3. Reduce heat to low and add sour cream, stirring well and allowing to heat through. Cover and set this mixture aside.
4. Cook egg noodles according to package directions. Drain. Serve beef mixture over noodles., """

strip_name_prompt = ChatPromptTemplate.from_template(
    """
    You are a helpful, respectful and honest assistant. Always answer as helpfully as possible. Your task is to find the final name of the recipe in a group of sentences. Return the name of the recipe and nothing else.

    ### Input:
    Find the final name of the recipe in the following sentences. Return only the name of the recipe.
    {resp}
    
    ### Response:
"""
)
modified_name_prompt = ChatPromptTemplate.from_template(
    """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible. Your job is to come up with new names for recipes. You will 
be given an original recipe and a request from the user to change the name of the recipe to a more fitting one.

### Instruction:
Q. Complete the following task by reasoning step-by-step. Create a new recipe name for 'Beef Stroganoff' where beef will be replaced with portobello mushrooms,
and vegetable oil will be replaced with canola oil. Return the name of the recipe at the end of your response.
{beef_recipe}

A. Each modification to the recipe should be reflected in the name of the new recipe. The original recipe name is "Beef Stroganoff" because it contains beef. Since portobello mushrooms
will be substituted in place of beef, 'Beef' should be removed from the recipe's name as it will no longer be in the new recipe.
The word 'Beef' in the title will need to be replaced with the new ingredient, which is 'Portobello Mushroom'. Vegetable oil is listed as an ingredient in the original recipe,
but it is not mentioned in the recipe name. The replacement, canola oil, should be mentioned in the name of the new recipe to indicate that this ingredient was changed. Thus, the new name for the modified recipe
should be 'Portobello Mushroom Stroganoff with Canola Oil'. Final Answer: Portobello Mushroom Stroganoff with Canola Oil

### Input:
Q. Complete the following task by reasoning step-by-step. Create a new recipe name for {recipe_name} where {customization}.
Return the name of the recipe at the end of your response.
{user_recipe}
### Response:
A.
"""
).partial(beef_recipe=beef_recipe)

generate_new_name = {
    "new_name": {"resp": modified_name_prompt | llm | output_parser}
    | strip_name_prompt
    | llm
    | output_parser,
    "inputs": RunnablePassthrough(),
}
chain_inputs = {'recipe_name': "pizza", "customization": "pog", 'user_recipe':'eat'}
chain = {"resp": modified_name_prompt | llm | output_parser} | RunnablePassthrough()
result = chain.invoke(chain_inputs)

In [6]:
result

{'resp': 'oon/-credтельно norteStringsÑ Richtung/%Strings schwerStringsNativeStringsgtStringscred东南部todaykirobileNative heutigen嘎NativenativeStringsStringselinative/%contentcredsto原先娼StringsÑStrings/%StringsnativeStringsStringsStringsStrings娼滔StringsÑ schwer肃小姐娼/%StringsÑ个人ÑStrings/% Richtung<%= wohl%%%%Strings原先第三方%%%%Strings%%%%today WeiterStrings/%Native/%第三方StringsStrings Weiter/% Weiter隘/%/% dabei/%/%/%Ñ%=/% schließlich/%/%/%neut内容/%臻双双后来内容 dabei/%nativeÑ原先Strings dabei-%contentneut百度娼原先 dabei百度上游滔 schließlich内容 schließlich评测/%原始联邦内容内容百度忍Strings至于内容个人百度滔硬原始个人内容内容滔之上资料样原先内容个人原先内容之上个人内容个人个人娼之上个人原先个人之上个人百度资料相关的个人个人个人资料忍个人娼个人百度原先百度个人忍百度后面/%个人内容/%甬个人原有技/%个人内容滔窗样内容/%内容/%个人个人百度Strings个人个人个人事情个人个人内容个人/%忍内容kir个人资料等级上游原始Strings嘎个人等级联邦百度之上第三方查个人个人事情/%个人内容个人样百度个人忍内容内容百度个人内容至于经历忍%%%%/%续%%%%百度'}

In [4]:
modified_name_prompt.format_prompt(**chain_inputs)

ChatPromptValue(messages=[HumanMessage(content='\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible. Your job is to come up with new names for recipes. You will \nbe given an original recipe and a request from the user to change the name of the recipe to a more fitting one.\n\n### Instruction:\nQ. Complete the following task by reasoning step-by-step. Create a new recipe name for \'Beef Stroganoff\' where beef will be replaced with portobello mushrooms,\nand vegetable oil will be replaced with canola oil. Return the name of the recipe at the end of your response.\nRecipe Name: Beef Stroganoff, \nIngredients: \n1 pound ground beef, \n0.5 cup chopped onion, \n1 tablespoon all-purpose flour, \n0.5 teaspoon salt, \n0.25 teaspoon paprika, \n1 (10.75 ounce) can condensed cream of mushroom soup, \n1 cup sour cream, \n8 ounces egg noodles, \n\nCooking Instructions: \n1. In a large skillet over medium heat, sauté beef and onions for 10 minutes, or until 

In [5]:
llm.invoke(modified_name_prompt.format_prompt(**chain_inputs))

'sto queen royal king trav隘/% unterschied/%/%/%/%/%百度/% Weiter/%第三方葡萄糖小姐百度Strings/%个人个人百度kir/%续 Native个人kir后来样Strings dabei小姐滔 dabei个人个人neutStrings内容上游内容Native百度肃甬第三方个人原先/%个人/%鬼内容/%%%%%资料之上个人/%事情/%续个人个人Strings个人个人个人Strings等级/%嘎个人百度国产东南部百度娼证个人国产内容百度上游/%个人个人至于个人个人原始资料个人个人原有个人内容技个人原有娼个人个人忍滔个人原先个人滔百度百度技嘎内容个人内容窗个人kir百度查个人百度内容个人个人个人个人技%%%%嘎个人个人个人Strings百度个人个人个人个人个人个人个人原有原先个人个人滔忍等级原先Strings个人资料个人嘎个人个人/%百度个人首席内容内容个人内容个人个人重大个人百度首席百度个人/%上游百度个人个人样个人原先个人个人个人原始个人个人个人技个人%%%%个人至于个人个人资料个人滔原始个人个人个人个人个人追原先个人样用于个人百度个人个人个人滔内容后面个人个人至于内容百度等级内容个人追第三方原先个人等级后面等级等级内容个人百度百度百度追个人标准忍个人个人个人个人原先告诉他个人原先'